Σε αυτό το notebook υλοποιούμε τον αλγόριθμο DGI για τα καλύτερα μοντέλα που προέκυψαν απο το προηγούμενο. Τα πειράματα για κάθε μοντέλο είναι δύο το πρώτο με χρήση του encoder και βάρη σε κατάσταση freeze και το δεύτερο με τα βάρη του encoder εκπαιδεύσιμα. Αρχικά φορτώνουμε το data object που αντιστοιχεί στο σενάριο imbalance καθώς αυτό έδωσε τα καλύτερα αποτελέσματα. Στην συνέχεια χρησιμοποιούμε τα δύο train functions που έιχαμε και στο προηγούμενο. Στην συνέχεια δημιουργούμε το μοντέλο για κάθε περίπτωση και το train function του DGI και το εκπαιδεύουμε για 200 εποχές. Μετά πέρνουμε το αντικείμενο encoder από το μοντέλο dgi και δημιουργούμε τον classifier. Στην συνέχεια τρέχουμε τα δύο σενάρια και αποθηκεύουμε τα αποτελέσματα για περεταίρω επεξεργασία.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Diplomatic2/Implementation")
!ls

 AGNN_1			      elliptic_txs_features.csv   GRL.ipynb
 AGNN_2			      FINAL_RESULTS		  Node2Vec
 AGNN_3			      GAT_1			  node_deg.obj
 DATA			      GAT_2			  node_d.obj
 data_balanced_1000.obj       GAT_2_1			  NoteBooks
 data_balanced_500.obj	      GAT_3			  project
 data_node_deg_features.obj   GAT_FL_1			  requirements.txt
 data.obj		      GAT_RELU_1		  SAGE_1
 DGI			      GAT_TEST			  SAGE_2
 elliptic_txs_classes.csv     GRL_2.ipynb		  SAGE_3
 elliptic_txs_edgelist.csv   'GRL_FINAL (4).ipynb'	  stellar.obj


In [ ]:
#PYTORCH INSTALL
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

In [ ]:
#LOAD DATA OBJECT
import pickle   
import torch
from torch_geometric.data import Data

fileObj = open('data.obj', 'rb')
data = pickle.load(fileObj)
fileObj.close()
print(data)

Data(x=[203769, 165], edge_index=[2, 234355], y=[203769], n_id=[203769], train_mask=[203769], val_mask=[203769], test_mask=[203769])


In [ ]:
#USE GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
data = data.to(device)


In [ ]:
#EDW NA DW ALLES PERIPTWSEIS GIA BATCHES!!!
#Epishs na dw th lista me tous geitones
from torch_geometric.loader import NeighborLoader

train_loader = NeighborLoader(data,num_neighbors=[-1, -1,-1], shuffle=True,batch_size=203769)

counter = 0
for batch in train_loader:

    print(len(batch.n_id))
    print(len(batch.y))
    print(type(batch))
    print(batch.test_mask)

203769
203769
<class 'torch_geometric.data.data.Data'>
tensor([False, False, False,  ..., False, False, False], device='cuda:0')


In [ ]:
#SCRIPT 1
#DHMIOURGIA MONTELOY SAGE KAI ANTISTOIXOU CLASSIFIER
from torch_geometric.nn import DeepGraphInfomax, SAGEConv
import torch.nn.functional as F

class GraphSAGE(torch.nn.Module):
  """GraphSAGE"""
  def __init__(self):
    super().__init__()
    
    self.sage1 = SAGEConv(165, 256)
    self.sage2 = SAGEConv(256, 128)
    self.sage3 = SAGEConv(128, 128)
    
  def forward(self, x, edge_index):

      h = self.sage1(x, edge_index)
      h = torch.relu(h)
      h = F.dropout(h, p=0.5, training=self.training)
      h = self.sage2(h, edge_index)
      h = torch.relu(h)
      h = F.dropout(h, p=0.2, training=self.training)
      h = self.sage3(h, edge_index)
      h = torch.relu(h)

      return h

class SAGE_Classifier(torch.nn.Module):
  """GraphSAGE"""
  def __init__(self,encoder):
    super().__init__()

    self.encoder = encoder
    #self.encoder.requires_grad_(False)
    self.linear = torch.nn.Linear(128,2)
    self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)
    
  def forward(self, x, edge_index):

      h = self.encoder(x, edge_index)
      h = self.linear(h)

      return h, F.softmax(h, dim=1)

In [ ]:
import copy

data2 = copy.deepcopy(data)

In [ ]:
#TRAI/VAL/TEST MASKS
train_index = data.train_mask.tolist()
val_index = data.val_mask.tolist()
test_index = data.test_mask.tolist()
print(type(train_index))
print(type(val_index))
print(type(test_index))

<class 'list'>
<class 'list'>
<class 'list'>


In [ ]:
#DEBUGGING CODE WORKS!!!!
model1 = GraphSAGE()
print(model1)
model2 = SAGE_Classifier(model1)
print(model2)
losses, val_losses, best_model, accuracy_v, auc_v, recall_score_v, precission_score_v, f1_v = train1(model2,1,False)
auc_score_GAT, acc_GAT, rec_GAT, pre_GAT, f1_GAT, conf_m_GAT = test(best_model,data)

In [ ]:
#DEBUGGING CODE WORKS
model3 = GATmodel()
print(model3)
model4 = GAT_Classifier(model3)
print(model4)
losses, val_losses, best_model, accuracy_v, auc_v, recall_score_v, precission_score_v, f1_v = train2(model4,1,False)
auc_score_GAT, acc_GAT, rec_GAT, pre_GAT, f1_GAT, conf_m_GAT = test(best_model,data)

In [ ]:
#SCRIPT 2
#GAT MODEL KAI ANTISTOIXOS CLASSIFIER
from torch_geometric.nn import DeepGraphInfomax, GATConv
import torch.nn.functional as F
import copy

class GATmodel(torch.nn.Module):
    def __init__(self):
        super(GATmodel, self).__init__()
        self.hid = 32
        self.in_head = 8
      
        self.conv1 = GATConv(165, self.hid, heads=self.in_head, dropout=0.6)
        self.conv = GATConv(self.hid*self.in_head, self.hid, heads=self.in_head,concat=False, dropout=0.6)
        #self.conv2 = GATConv(self.hid*self.in_head, self.hid, heads=self.in_head, dropout=0.6)
       
          
    def forward(self,x, edge_index):
        
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv(x, edge_index)
        x = F.elu(x)
        
        return x   


class GAT_Classifier(torch.nn.Module):
  def __init__(self,encoder):
    super().__init__()

    self.encoder = encoder
    #self.encoder.requires_grad_(False)
    self.linear = torch.nn.Linear(32,2)
    self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)
    
  def forward(self, x, edge_index):
#NA BALW DROP OUT SE OLA
      h = self.encoder(x, edge_index)
      h = self.linear(h)

      return h, F.softmax(h, dim=1)

In [ ]:
#SCRIPT 3
#AGNN MODEL KAI ANTISTOIXOS CLASSIFIER
from torch_geometric.nn import DeepGraphInfomax, AGNNConv
import torch.nn.functional as F

class AGNNmodel(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.hidden = 64

        self.lin1 = torch.nn.Linear(165, self.hidden)
        self.prop1 = AGNNConv(requires_grad=False)
        self.prop2 = AGNNConv(requires_grad=True)


    def forward(self, x, edge_index):

        x = F.dropout(x, training=self.training)
        x = F.relu(self.lin1(x))
        x = self.prop1(x, edge_index)
        x = self.prop2(x, edge_index)

        return x 

class AGNN_Classifier(torch.nn.Module):
  def __init__(self,encoder):
    super().__init__()

    self.encoder = encoder
    #self.encoder.requires_grad_(False)
    self.linear = torch.nn.Linear(64,2)
    self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)
    
  def forward(self, x, edge_index):

      h = self.encoder(x, edge_index)
      h = self.linear(h)

      return h, F.softmax(h, dim=1)        

In [ ]:
#SCRIPT 4
#METRICS
!pip install torchmetrics
from torchmetrics.classification import AUROC
from torchmetrics.classification import BinaryPrecision
from torchmetrics.classification import BinaryRecall
from torchmetrics.classification import BinaryConfusionMatrix
from torchmetrics.classification import MulticlassRecall
from torchmetrics.classification import MulticlassPrecision
from torchmetrics.classification import MulticlassAUROC
from torchmetrics.classification import BinaryF1Score
from torchmetrics.classification import MulticlassF1Score
from torchmetrics.classification import Accuracy

confmat = BinaryConfusionMatrix()
recall = MulticlassRecall(num_classes=2, average=None)
precision = MulticlassPrecision(num_classes=2, average=None)
aucroc = MulticlassAUROC(task="multiclass", num_classes=2, thresholds=None)
#f1 = BinaryF1Score(average=None)
f12 = MulticlassF1Score(num_classes=2, average=None)
accuracy = Accuracy(task="multiclass", num_classes=2)

def conf_matrix(pred_y, y):
  #confmat = BinaryConfusionMatrix().to(device)

  return confmat(pred_y, y)

def brecall(pred_y, y):
  #recall = BinaryRecall().to(device)
  #recall = MulticlassRecall(num_classes=2, average=None).to(device)

  return recall(pred_y, y)

def bprecision(pred_y, y):
  #precision = MulticlassPrecision(num_classes=2, average=None).to(device)

  return precision(pred_y, y)

def auc_roc(pred_y, y):
  #aucroc = AUROC(task="binary").to(device)
  #print('AUC ROC',pred_y[:10])
  #print('AUC ROC',y.shape)
  #aucroc = MulticlassAUROC(task="multiclass", num_classes=2).to(device)

  return aucroc(pred_y, y)

def f1score(pred_y, y):
  #metric = BinaryF1Score(average=None).to(device)

  return f12(pred_y, y)


def accuracy_score(pred_y, y):
    
    return accuracy(pred_y,y)

In [ ]:
#SCRIPT 5
#TRAIN ONE EPOCH
def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def train_step(data):
        # Sets model to TRAIN mode
        model.train()
        # Makes predictions
        logit, out = model(data.x,data.edge_index)
        # Computes loss
        loss = loss_fn(logit[data.train_mask], data.y[data.train_mask])
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # Returns the loss
        return loss.item(), out.tolist()
    
    # Returns the function that will be called inside the train loop
    return train_step

# Creates the train_step function for our model, loss function and optimizer
#train_step = make_train_step(model, loss_fn, optimizer)
#losses = []

# For each epoch...
#for epoch in range(n_epochs):
    # Performs one train step and returns the corresponding loss
    #loss = train_step(x_train_tensor, y_train_tensor)
    #losses.append(loss)
    

In [ ]:
#SCRIPT 4
#FUNCTION TO RETURN METRICS
import numpy as np

def metrics_computation(truth,pred_probs):
  print('METRICS ',len(pred_probs),len(truth),type(truth),type(pred_probs))
  #print(pred_probs[0],truth[0])
  #print(type(argmax_res),argmax_res.shape)
  #FOR AUCROC
  probs_tensor = torch.from_numpy(pred_probs)
  #print('tensor probs ',probs_tensor.shape)
  #FOR THE REST OF METRICS
  argmax_res = torch.argmax(probs_tensor,dim=1)
  #print('argmax ',argmax_res.shape)
  #FOR AUC AND REST TRYTH TENSOR
  truth_tensor = np.array(truth)
  truth_tensor = torch.from_numpy(truth_tensor)
  #print('TRUTH ',truth_tensor.shape)

  #COMPUTE AUC ROC SCORE
  auc_score = auc_roc(probs_tensor,truth_tensor)
  #print('AUC SCORE ', auc_score)

  #COMPUTE ACCURACY
  acc = accuracy_score(argmax_res,truth_tensor)
  #print('ACCURACY SCORE',acc)

  #COMPUTE RECALL FOR BOTH CLASSES
  rec = brecall(argmax_res,truth_tensor)
  #print('RECALL SCORE ',rec) 

  #COMPUTE PRECISSION FOR BOTH CLASSES 
  pre = bprecision(argmax_res,truth_tensor)
  #print('PRECISSION SCORE ',pre)

  #COMPUTE F1 SCORE FOR BOTH CLASSES
  f1_score = f1score(argmax_res,truth_tensor)
  #print('F1 SCORE ',f1_score)

  #COMPUTE CONFUSION MATRIX
  conf_m = conf_matrix(argmax_res,truth_tensor)
  #print('CONFUSION MATRIX ', conf_m)

  return auc_score, acc, rec, pre, f1_score, conf_m



In [ ]:
#SCRIPT 5
#TRAIN FUNCTION FOR GAT/AGNN
#TRAIN WITH DATA OBJECT
def train2(model,epochs,weigths):

  if weigths == True:

    print('WEIGTHS = ',weigths)
    criterion = torch.nn.CrossEntropyLoss(weight=imbalance)
  else:
    criterion = torch.nn.CrossEntropyLoss()

  train_step = make_train_step(model, criterion, model.optimizer) 

  losses = []
  val_losses = []
  best_model = copy.deepcopy(model)
  best_val_loss = 10000

  #METRICS VAL
  accuracy_v = []
  auc_v = []
  recall_score_v = []
  precission_score_v = []
  f1_v = []
  
  print('WORKING WITH TRAIN2')
  for epoch in range(epochs):
    
        
        #NA TO GYRISW SE SOFTMAX ANTI LOG
        loss, probs_train = train_step(data)
        losses.append(loss)

        #print('TRAIN TENSOR ',type(probs_train),len(probs_train),probs_train[0])
        #print('PROBS TRAIN',probs_train[0])
        #tens = torch.tensor([0,1,1,1,0,0])
        #pred_train  = argmax_list(probs_train)

        train_truth = data2.y[train_index].tolist()
        #print('train truth', type(train_truth))

        #print('SIZE ',len(pred),type(pred))
        #pred = torch.tensor(pred)
        #print('SIZE ',len(pred),type(pred),pred.dtype)
        #pred = pred[train_index]
        #print('SIZE ',len(pred),type(pred),pred,pred.dtype)
        #print('B ',len(b),type(b),b.dtype)
        #print('LOSS',loss)
        
    #print('TEST ',type(probs),len(probs),probs[0])
    
        with torch.no_grad():
            
            model.eval()

            logit, out = model(data.x,data.edge_index)
            val_loss = criterion(logit[data.val_mask], data.y[data.val_mask])
            val_losses.append(val_loss.item())

            #print('PROBS VAL ',out.tolist()[0])

            val_probs = out.tolist()
            val_probs = np.array(val_probs)
            val_probs = val_probs[val_index]

            probs_train = np.array(probs_train)
            probs_train = probs_train[train_index]
            

            val_truth = data2.y[val_index].tolist()
            auc_score_val, acc_val, rec_val, pre_val, f1_score_val, conf_m_val = metrics_computation(val_truth,val_probs)
            auc_score_train, acc_train, rec_train, pre_train, f1_score_train, conf_m_train = metrics_computation(train_truth,probs_train)
            
            print('AUC VAL',auc_score_val,' ACC VAL ',acc_val,' RECALL VAL ',rec_val, 'PRECI VAL ', pre_val,' F1 VAL ', f1_score_val,' CM VAL ',conf_m_val)
            print('AUC TRAIN',auc_score_train,' ACC TRAIN ',acc_train,' RECALL TRAIN ',rec_train, 'PRECI TRAIN ', pre_train,' F1 TRAIN ', f1_score_train,' CM TRAIN ',conf_m_train)

        accuracy_v.append(acc_val.item())
        auc_v.append(auc_score_val.item())
        recall_score_v.append(rec_val.tolist())
        precission_score_v.append(pre_val.tolist())
        f1_v.append(f1_score_val.tolist())

        if val_loss < best_val_loss:
          print('LOSSES',val_loss.item(),best_val_loss)
          best_model = copy.deepcopy(model)
          best_val_loss = copy.deepcopy(val_loss.item())
          print('NEW BEST///')     
  return losses, val_losses, best_model, accuracy_v, auc_v, recall_score_v, precission_score_v, f1_v 

In [ ]:
#SCRIPT 5
#TRAIN FUNCTION FOR SAGE
#TRAIN WITH LOADER
def train1(model,epochs,weigths):

  if weigths == True:

    print('WEIGTHS = ',weigths)
    criterion = torch.nn.CrossEntropyLoss(weight=imbalance)
  else:
    criterion = torch.nn.CrossEntropyLoss()

  train_step = make_train_step(model, criterion, model.optimizer) 

  losses = []
  val_losses = []
  best_model = copy.deepcopy(model)
  best_val_loss = 10000

  #METRICS VAL
  accuracy_v = []
  auc_v = []
  recall_score_v = []
  precission_score_v = []
  f1_v = []
  

  for epoch in range(epochs):
    for batch in train_loader:
        batch = batch.to(device)
        #NA TO GYRISW SE SOFTMAX ANTI LOG
        loss, probs_train = train_step(batch)
        losses.append(loss)

        #print('TRAIN TENSOR ',type(probs_train),len(probs_train),probs_train[0])
        #print('PROBS TRAIN',probs_train[0])
        #tens = torch.tensor([0,1,1,1,0,0])
        #pred_train  = argmax_list(probs_train)

        train_truth = data2.y[train_index].tolist()
        #print('train truth', type(train_truth))

        #print('SIZE ',len(pred),type(pred))
        #pred = torch.tensor(pred)
        #print('SIZE ',len(pred),type(pred),pred.dtype)
        #pred = pred[train_index]
        #print('SIZE ',len(pred),type(pred),pred,pred.dtype)
        #print('B ',len(b),type(b),b.dtype)
        #print('LOSS',loss)
        
    #print('TEST ',type(probs),len(probs),probs[0])
    
    with torch.no_grad():
        for batch in train_loader:
            batch = batch.to(device)
            
            model.eval()

            logit, out = model(batch.x,batch.edge_index)
            val_loss = criterion(logit[batch.val_mask], batch.y[batch.val_mask])
            val_losses.append(val_loss.item())

            #print('PROBS VAL ',out.tolist()[0])

            val_probs = out.tolist()
            val_probs = np.array(val_probs)
            val_probs = val_probs[val_index]

            probs_train = np.array(probs_train)
            probs_train = probs_train[train_index]
            

            val_truth = data2.y[val_index].tolist()
            auc_score_val, acc_val, rec_val, pre_val, f1_score_val, conf_m_val = metrics_computation(val_truth,val_probs)
            auc_score_train, acc_train, rec_train, pre_train, f1_score_train, conf_m_train = metrics_computation(train_truth,probs_train)
            
            print('AUC VAL',auc_score_val,' ACC VAL ',acc_val,' RECALL VAL ',rec_val, 'PRECI VAL ', pre_val,' F1 VAL ', f1_score_val)
            print('AUC TRAIN',auc_score_train,' ACC TRAIN ',acc_train,' RECALL TRAIN ',rec_train, 'PRECI TRAIN ', pre_train,' F1 TRAIN ', f1_score_train)

        accuracy_v.append(acc_val.item())
        auc_v.append(auc_score_val.item())
        recall_score_v.append(rec_val.tolist())
        precission_score_v.append(pre_val.tolist())
        f1_v.append(f1_score_val.tolist())

        if val_loss < best_val_loss:
          print('LOSSES',val_loss.item(),best_val_loss)
          best_model = copy.deepcopy(model)
          best_val_loss = copy.deepcopy(val_loss.item())
          print('NEW BEST///')     
  return losses, val_losses, best_model, accuracy_v, auc_v, recall_score_v, precission_score_v, f1_v

In [ ]:
#SCRIPT 5
#TEST FUNCTION
def test(model, data):
    """Evaluate the model on test set and print the accuracy score."""
    with torch.no_grad():
      model.eval()
      _, out = model(data.x, data.edge_index)
      test_probs = out.tolist()
      test_probs = np.array(test_probs)
      test_probs = test_probs[test_index]

      test_truth = data2.y[test_index].tolist()

      auc_score_test, acc_test, rec_test, pre_test, f1_score_test, conf_m_test = metrics_computation(test_truth,test_probs)



    return auc_score_test.item(), acc_test.item(), rec_test.tolist(), pre_test.tolist(), f1_score_test.tolist(), conf_m_test.tolist()

In [ ]:
#SCRIPT 6
#TRAIN ENCODER WITH DGI
#CORRUPTION TO CREATE ANOTHER GRAPH
#SUMMARY FUNCTION IS SIGMOID OF POS_Z.MEAN() DOUBLE CHECKED
#encoder_SAGE = GraphSAGE()
#print(encoder_SAGE)
#encoder_GAT = GATmodel()
#print(encoder_GAT)
#encoder_AGNN = AGNNmodel()
#print(encoder_AGNN)

def corruption(x, edge_index):
    return x[torch.randperm(x.size(0))], edge_index

model_dgi = DeepGraphInfomax(hidden_channels=128, encoder=GraphSAGE(),summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),corruption=corruption).to(device)

optimizer = torch.optim.Adam(model_dgi.parameters(), lr=0.001)

def train_dgi():
    model_dgi.train()
    optimizer.zero_grad()
    pos_z, neg_z, summary = model_dgi(data.x, data.edge_index)
    loss = model_dgi.loss(pos_z, neg_z, summary)
    loss.backward()
    optimizer.step()
    return loss.item()

In [ ]:
os.chdir("/content/drive/MyDrive/Diplomatic2/Implementation/FINAL_RESULTS/DGI_EMB")
!ls

AGNN_DGI.emb  GAT_DGI.emb


In [ ]:
for epoch in range(1,200):
  loss = train_dgi()
  print(loss)

1.3983526229858398
1.3878659009933472
1.4153600931167603
1.3775641918182373
1.38112211227417
1.3857982158660889
1.376237154006958
1.3689396381378174
1.366485357284546
1.3647058010101318
1.3595751523971558
1.3491764068603516
1.343616247177124
1.3358144760131836
1.3205673694610596
1.3122589588165283
1.2965821027755737
1.2865231037139893
1.2630763053894043
1.2534689903259277
1.2327702045440674
1.2112398147583008
1.2071101665496826
1.1937062740325928
1.1728644371032715
1.1587374210357666
1.1538641452789307
1.1343045234680176
1.125657558441162
1.117638111114502
1.1021983623504639
1.0927391052246094
1.0806639194488525
1.06976318359375
1.053657054901123
1.0516541004180908
1.0409750938415527
1.0256233215332031
1.0132330656051636
1.0037477016448975
0.9963935613632202
0.9838992953300476
0.9734538793563843
0.9661258459091187
0.9527742862701416
0.9422775506973267
0.9392518997192383
0.9249292612075806
0.9237951040267944
0.9094126224517822
0.8957882523536682
0.8838517665863037
0.8810778260231018
0.8

In [ ]:
with torch.no_grad():
 model_dgi.encoder.eval()
 tens = model_dgi.encoder(data.x, data.edge_index)

In [ ]:
print(type(tens))
print(tens.shape)

<class 'torch.Tensor'>
torch.Size([203769, 128])


In [ ]:
gat_emb = tens.tolist()


In [ ]:
import numpy as np

gat_emb = np.array(gat_emb)

In [ ]:
import pickle
import os

os.chdir("/content/drive/MyDrive/Diplomatic2/Implementation/FINAL_RESULTS/DGI_EMB")
!ls
architecture  = 'SAGE_DGI.emb'
file2save_everything = open(architecture,'wb')
pickle.dump(gat_emb,file2save_everything)
file2save_everything.close()


AGNN_DGI.emb  GAT_DGI.emb


In [ ]:
encode_obj2 = copy.deepcopy(model_dgi.encoder)

In [ ]:
AGNN_Class2 = AGNN_Classifier(encode_obj2).to(device)
print(AGNN_Class2)

In [ ]:
#SCRIPT 7 
#TRAIN CLASSIFIERS WITH AND WITHOUT FREEZED PARAMETERS
losses, val_losses, best_model, accuracy_v, auc_v, recall_score_v, precission_score_v, f1_v = train2(AGNN_Class2,200,False)
auc_score_GAT, acc_GAT, rec_GAT, pre_GAT, f1_GAT, conf_m_GAT = test(best_model,data)

In [ ]:
print(auc_score_GAT, acc_GAT, rec_GAT, pre_GAT, f1_GAT, conf_m_GAT)

0.9263032674789429 0.9358312487602234 [0.5324165225028992, 0.9745465517044067] [0.6674876809120178, 0.9559813737869263] [0.5923497080802917, 0.965174674987793] [[1355, 1190], [675, 25844]]


In [ ]:
file2save_everything = open('agnn_train','wb')
#pickle.dump(best_model, file2save_everything)
#SAVE TRAIN DATA
pickle.dump(losses, file2save_everything)
pickle.dump(val_losses, file2save_everything)
pickle.dump(accuracy_v, file2save_everything)
pickle.dump(auc_v, file2save_everything)
pickle.dump(recall_score_v, file2save_everything)
pickle.dump(precission_score_v, file2save_everything)
pickle.dump(f1_v, file2save_everything)
#SAVE TEST DATA
pickle.dump(auc_score_GAT, file2save_everything)
pickle.dump(acc_GAT, file2save_everything)
pickle.dump(rec_GAT, file2save_everything)
pickle.dump(pre_GAT, file2save_everything)
pickle.dump(f1_GAT, file2save_everything)
pickle.dump(conf_m_GAT, file2save_everything)

file2save_everything.close()